In [18]:
import re
import requests
import pandas as pd
from urllib import parse
from bs4 import BeautifulSoup
from datetime import datetime

In [3]:
keyword = input('검색 키워드 : ')

검색 키워드 : 노트북


In [23]:
url = 'https://www.coupang.com/np/search'
params = {
    'q':keyword
}

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
}

# 보통 안되면 유저에이전트 그 다음 레퍼러 그 다음 쿠키.

- 제품명
- 상세 링크
- 판매가

In [12]:
# base_url : ul#productList
# 제품명 : div.name : text
# 상세 링크 : a : href 속성
# 판매가 : strong.price-value : text


# #\31 681782369 > a > dl > dd > div > div.name

In [13]:
res = requests.get(url, params=params, headers=headers)

In [16]:
if res.status_code == 200:
    soup = BeautifulSoup(res.text)
    
#     name = soup.select_one('div.name').text
#     link = soup.select_one('div.name').get('href')
#     price = soup.select_one('strong.price-value').text
    
# name, link, price
    
    item_list = soup.select('ul#productList li')
#     print(len(item_list))
    
    result_list = [] # 조회 결과를 담을 리스트
    
    for item in item_list:
        item_name = item.select_one('div.name').text.strip()
        link = item.select_one('a').get('href')
        
        price = item.select_one('strong.price-value').text.strip()
        print(item_name, link, price, sep='\n')
        break
    

삼성전자 갤럭시북 이온 NT950XCJ-K78A (i7-10510U 39.6cm), 포함, NVMe 256GB + NVMe 256GB, 16GB
/vp/products/1137914312?itemId=2517400150&vendorItemId=70510368267&sourceType=srp_product_ads
1,959,000


In [22]:
# 가격이 없는 제품이 있을 때

if res.status_code == 200:
    soup = BeautifulSoup(res.text)
    
#     name = soup.select_one('div.name').text
#     link = soup.select_one('div.name').get('href')
#     price = soup.select_one('strong.price-value').text
    
# name, link, price
    
    item_list = soup.select('ul#productList li')
#     print(len(item_list))
    
    result_list = [] # 조회 결과를 담을 리스트
    error_cnt = 0
    
    base_url = 'https://www.coupang.com/'
    for item in item_list:
        try:
            item_name = item.select_one('div.name').text.strip()
            link = item.select_one('a').get('href')
            link = parse.urljoin(base_url, link)
            
            # 가격에서 쉼표 구분자 제거하기.
            price = item.select_one('strong.price-value').text.strip()
            price = ''.join(price.split(','))
            
            result_list.append((item_name, link, price))
            
        except Exception as e:
            print(e)
            error_cnt += 1
            
    # 판다스를 이용해 csv / excel 파일로 저장.
    filename = '쿠팡_{keyword}_조회결과_{date}'.format(keyword=keyword, date=datetime.now().strftime('%Y-%m-%d'))
    
    df = pd.DataFrame(result_list, columns=['상품명', '링크', '가격'])
    df.to_csv(filename+'.csv', index=False, encoding='UTF-8')
    df.to_excel(filename+'.xls', index=False)

In [ ]:
# 쿠팡 전체 페이지 크롤링해보기.
